In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
from scipy.optimize import curve_fit
import math
import torch

In [2]:
def softplus(x, beta):
    """Softplus function with sharpness parameter."""
    return (1.0 / beta) * np.log(1 + np.exp(beta * x))

def combined_function(x, alpha, beta, gamma, phi, sharpness):
    """Modified function combining linear and softplus terms."""
    return alpha * (x - gamma) + beta * softplus(x - gamma, sharpness) + phi


In [3]:
data = torch.load('./data/pReLU_vout.data').numpy()
data.shape

(401, 9999)

In [4]:
x = np.linspace(-2, 2, 401)
result = []

for k in range(data.shape[1]):
    y = data[:, k]
    
    popt, _ = curve_fit(combined_function, x, y, maxfev=1000000, p0=[0.2, 1., 0., 0., 10.], bounds=([0, 0, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf, np.inf]))
    alpha, beta, gamma, phi, sharpness = popt
    result.append([alpha, beta, gamma, phi, sharpness])

    plt.figure()
    plt.plot(x, y, color='green')
    plt.plot(x, combined_function(x, *popt), color='red') 
    plt.savefig(f'./prelu_figures/{k}.pdf', format='pdf', bbox_inches='tight')
    plt.close()

In [5]:
a = torch.tensor(result)
a.shape

torch.Size([9999, 5])

In [6]:
torch.save(a, './data/prelu_param.data')